Create an API request from OpenWeatherMap for a weather data for our desired cities. 

In [ ]:
import pandas as pd
import requests
from requests.exceptions import HTTPError, ConnectionError, Timeout, RequestException
from api_keys import API_key

cities= ["Berlin", "Hamburg", "Munich"]
weather_set = [] 

for city in cities:
    try:
        response=requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        data=response.json()
        weather_all=pd.json_normalize(data["list"])
        weather_all["city"]=city
        weather_set.append(weather_all)
    
    except HTTPError as http_err:
        print(f"HTTP error occured for {city} : {http_err}")
    except ConnectionError as conn_err:
        print(f"Connection error occured for {city} : {conn_err}")
    except Timeout as timeout:
        print(f"Timeout error occurred for {city} : {timeout}")
    except RequestException as req_error:
        print(f"Request error occured for {city} : {req_error}")
    except Exception as exception:
        print(f"An unexpected error occurred for {city} : {exception}")
            
weather_fetch=pd.concat(weather_set, ignore_index=True)

weather_fetch=weather_fetch.drop(["rain.3h","sys.pod","main.sea_level", "main.grnd_level", "main.temp_kf", "wind.gust", "wind.deg", "clouds.all", "wind.speed", "main.humidity", "main.pressure","dt", "pop", "visibility", "main.feels_like"], axis=1)

weather_fetch["city_id"]=""
weather_fetch.loc[weather_fetch["city"] == "Berlin", "city_id"]=1
weather_fetch.loc[weather_fetch["city"] == "Hamburg", "city_id"]=2
weather_fetch.loc[weather_fetch["city"] == "Munich", "city_id"]=3
weather_fetch["conditions"]=""

def conditions(row):
    if "Clear" in row["weather"][0]["main"]:
        return "Clear"
    elif "Clouds" in row["weather"][0]["main"]:
        return "Clouds"
    else:
        return "Rain"
conditions

weather_fetch["conditions"]=weather_fetch.apply(conditions, axis=1)

weather_fetch=weather_fetch.drop(["weather"], axis=1)
weather_fetch=weather_fetch.rename(columns={"conditions": "weather"})
weather_fetch=weather_fetch.rename(columns={"main.temp": "temperature"})
weather_fetch=weather_fetch.rename(columns={"main.temp_min": "temp_min"})
weather_fetch=weather_fetch.rename(columns={"main.temp_max": "temp_max"})
weather_fetch
def table_to_sql():
    schema = ""
    host = ""
    user = ""
    password = ""
    port = 3306 
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    weather_fetch.to_sql("weather", if_exists="replace", con=connection_string, index=False)
table_to_sql()
weather_fetch